# Build Analysis Data

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange
from datetime import date
from multiprocess.pool import Pool


storage = "/Volumes/easystore/Drones/"
calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
flight_details = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/gpx-with-census-data.csv", 
                chunksize=100000, 
                dtype=str
            ),
            desc="Loading data",
        )
    ]
)
flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/all-flights-manifest.csv", 
                chunksize=100000, 
                dtype=str
            ),
            desc="Loading data",
        )
    ]
)

cv_blocks = pd.read_csv('../../data/outputs/outputs_cv-blocks-geometry.csv',dtype='str')

# flight_data = pd.read_csv("/Volumes/easystore/Drones/compiled-flight-data-gpx.csv",dtype=str)
# flight_data.head()

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

## Standardize Analysis
- Calls for Serice are from 2021-07-01 until 2023-09-01
- Drones Flights are from 2021-04-01 until 2023-09-05

### Flights and Flight Details

In [2]:
cvpd_flights = flights[flights['department'] == 'cvpd'].copy()
cvpd_flights["date"] = pd.to_datetime(cvpd_flights["date"])

mask = (cvpd_flights['date'] >= '2021-07-01 00:00') & (cvpd_flights['date'] <= '2023-09-02 00:00')

cvpd_flights = cvpd_flights[mask].sort_values('date').copy()
cvpd_flights['clean-type'] = cvpd_flights['type'].str.lower()
cvpd_flights['clean-type'] = cvpd_flights['clean-type'].apply(lambda x: str(x).replace('  ','').strip())
cvpd_flights["match"] = cvpd_flights["incident_id"].astype(str).apply(lambda x: x.strip().replace(' ','').replace('L','0').replace('l','0')[-5:])

cvpd_flights.to_csv('../../data/outputs/outputs_cv-flight-manifest.csv',index=False)

In [3]:
flight_details["GEOID20"] = flight_details["geoid"].astype(str)
flight_details["len"] = flight_details["GEOID20"].apply(lambda x: len(x))
flight_details.loc[flight_details["len"] == 14, "GEOID20"] = "0" + flight_details["GEOID20"]
flight_details["len"] = flight_details["GEOID20"].apply(lambda x: len(x))


### Calls for Service

In [4]:
calls_for_service['len'] = calls_for_service['Full FIPS (block)'].astype(str).apply(lambda x: len(x))

calls_for_service = calls_for_service[calls_for_service['len']==15].copy()
calls_for_service['GEOID20'] = calls_for_service['Full FIPS (block)']
calls_for_service["match"] = (
    calls_for_service["Incident No."].astype(str).apply(lambda x: x.strip().replace(' ','').replace('l','0').replace('L','0')[-5:])
)
calls_for_service['clean-type'] = calls_for_service['Description'].str.lower()
calls_for_service['clean-type'] = calls_for_service['clean-type'].apply(lambda x: str(x).replace('  ','').strip())
calls_for_service.to_csv('../../data/outputs/outputs_cfs-data.csv',index=False)

## Group by Block

### Call for Service

In [5]:
call_for_service_block_count = calls_for_service.groupby(["GEOID20"]).count()
call_for_service_block_count = call_for_service_block_count.reset_index()[
    ["GEOID20", "Incident No."]
]
call_for_service_block_count.columns = ["GEOID20", "call_count"]
call_for_service_block_count.head()

,GEOID20,call_count
0,060250102002026,1
1,060650306022000,2
2,060650445182017,1
3,060650451232003,1
4,060650453021004,2


In [6]:
call_for_service_block_count_with_shps = pd.merge(
    cv_blocks, call_for_service_block_count, how="left", on=["GEOID20"]
)

In [7]:
call_for_service_block_count_with_shps["GEOID20"].apply(
    lambda x: len(x)
).drop_duplicates()


0    15
Name: GEOID20, dtype: int64

In [8]:
call_for_service_block_count_with_shps[
    "call_count"
] = call_for_service_block_count_with_shps["call_count"].replace(np.nan, 0)

call_for_service_block_count_with_shps['weight'] = call_for_service_block_count_with_shps['call_count'].astype(int)/call_for_service_block_count_with_shps['POP20'].astype(int)

call_for_service_block_count_with_shps[
    "weight"
] = call_for_service_block_count_with_shps["weight"].replace(np.nan, 0)
call_for_service_block_count_with_shps[
    "weight"
] = call_for_service_block_count_with_shps["weight"].replace(np.inf, 0)


In [9]:
call_for_service_block_count_with_shps.to_csv('../../data/outputs/outputs_cfs-data-with-shps.csv',index=False)

### Drone Seconds in Block

In [10]:
flight_details = flight_details[flight_details['id'].isin(cvpd_flights['id'])].copy()
flight_details.head()

,id,type,incident_id,address_map,sequence,longitude,latitude,altitude,success,geoid,...,block_group,tract,county,state,county_name,state_name,population,housing,GEOID20,len
2194017,6e43fee7c42973b78794b68f37d2d63a,Weapon Threat,CVL63747,Broadway/ H St.,2021-08-01 03:04:10+00:00,-117.0827,32.64,22.139286,True,60730123021013,...,1,12302,73,6,San Diego County,California,306,130,060730123021013,15
2194018,6e43fee7c42973b78794b68f37d2d63a,Weapon Threat,CVL63747,Broadway/ H St.,2021-08-01 03:04:11+00:00,-117.0827,32.64,22.139286,True,60730123021013,...,1,12302,73,6,San Diego County,California,306,130,060730123021013,15
2194019,6e43fee7c42973b78794b68f37d2d63a,Weapon Threat,CVL63747,Broadway/ H St.,2021-08-01 03:06:13+00:00,-117.0827,32.64,22.239286,True,60730123021013,...,1,12302,73,6,San Diego County,California,306,130,060730123021013,15
2194020,6e43fee7c42973b78794b68f37d2d63a,Weapon Threat,CVL63747,Broadway/ H St.,2021-08-01 03:06:14+00:00,-117.0827,32.64,22.339286,True,60730123021013,...,1,12302,73,6,San Diego County,California,306,130,060730123021013,15
2194021,6e43fee7c42973b78794b68f37d2d63a,Weapon Threat,CVL63747,Broadway/ H St.,2021-08-01 03:06:14+00:00,-117.0827,32.64,22.439286,True,60730123021013,...,1,12302,73,6,San Diego County,California,306,130,060730123021013,15


In [11]:
times = pd.to_datetime(flight_details["sequence"])
grouped_by_block_and_second = flight_details.groupby(
    ["id","GEOID20", times.dt.hour, times.dt.minute, times.dt.second]
).count()
grouped_by_block_and_second.index.names = ["id","GEOID20", "hour", "minute", "second"]
grouped_by_block_and_second = grouped_by_block_and_second.reset_index()[
    ["id","GEOID20", "hour", "minute", "second", "type"]
]
grouped_by_block_and_second.columns = ["id","GEOID20", "hour", "minute", "second", "count"]


In [12]:
per_flight_block_seconds = grouped_by_block_and_second.groupby(['id','GEOID20']).count()
per_flight_block_seconds = per_flight_block_seconds.reset_index()[["id","GEOID20", "hour"]]
per_flight_block_seconds.columns = ["id","GEOID20", "seconds"]
per_flight_block_seconds.head()
per_flight_block_seconds = pd.merge(
    per_flight_block_seconds, cv_blocks, how="left", on=["GEOID20"]
)

In [13]:
per_flight_block_seconds.to_csv('../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv',index=False)

In [14]:
unique_seconds_in_block = grouped_by_block_and_second.groupby(["GEOID20"]).count()
unique_seconds_in_block = unique_seconds_in_block.reset_index()[["GEOID20", "hour"]]
unique_seconds_in_block.columns = ["GEOID20", "seconds"]
unique_seconds_in_block.head()


,GEOID20,seconds
0,060730032041009,32
1,060730032041012,35
2,060730032041013,5525
3,060730032041014,2675
4,060730032041015,217


In [15]:
unique_seconds_in_block_with_shps = pd.merge(
    cv_blocks, unique_seconds_in_block, how="left", on=["GEOID20"]
)
unique_seconds_in_block_with_shps["seconds"] = unique_seconds_in_block_with_shps[
    "seconds"
].replace(np.nan, 0)


In [16]:
unique_seconds_in_block_with_shps['ALAND20'] = unique_seconds_in_block_with_shps['ALAND20'].astype(int)

In [17]:
unique_seconds_in_block_with_shps["weight"] = (
    unique_seconds_in_block_with_shps["seconds"]
    / unique_seconds_in_block_with_shps["ALAND20"]
)

In [21]:

unique_seconds_in_block_with_shps = pd.merge(unique_seconds_in_block_with_shps,call_for_service_block_count_with_shps[['GEOID20','call_count']],how='left',on='GEOID20')

In [22]:
unique_seconds_in_block_with_shps.to_csv('../../data/outputs/outputs_drone-seconds-block-count.csv',index=False)

In [23]:
unique_seconds_in_block_with_shps

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry,len,seconds,weight,call_count
0,6,73,12600,1005,060730126001005,Block 1005,G5040,U,78661.0,U,...,0,32.6326934,-117.0972764,0,1,"POLYGON ((-117.09848 32.636082, -117.098118 32...",15,3808.0,0.187882,0.0
1,6,73,12600,1000,060730126001000,Block 1000,G5040,U,78661.0,U,...,0,32.6327645,-117.0968738,0,1,"POLYGON ((-117.098419 32.636099, -117.098382 3...",15,7313.0,0.177212,0.0
2,6,73,12600,1001,060730126001001,Block 1001,G5040,U,78661.0,U,...,0,32.627561,-117.0948246,0,1,"POLYGON ((-117.095692 32.629235, -117.095466 3...",15,3007.0,0.156135,0.0
3,6,73,12600,1002,060730126001002,Block 1002,G5040,U,78661.0,U,...,0,32.6279724,-117.0932719,310,656,"POLYGON ((-117.095251 32.629345, -117.095177 3...",15,24035.0,0.230868,1170.0
4,6,73,12600,1003,060730126001003,Block 1003,G5040,U,78661.0,U,...,0,32.6284473,-117.0915697,39,94,"POLYGON ((-117.092611 32.630074, -117.092519 3...",15,7898.0,0.248084,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,6,73,10016,1000,060730100161000,Block 1000,G5040,R,NaN,NaN,...,0,32.5937593,-116.9514464,0,1,"POLYGON ((-116.966156 32.589925, -116.965916 3...",15,453.0,0.000431,0.0
1955,6,73,10016,1002,060730100161002,Block 1002,G5040,R,NaN,NaN,...,0,32.5850417,-116.9902654,0,1,"POLYGON ((-117.005217 32.592376, -117.005127 3...",15,41.0,0.000020,0.0
1956,6,73,13322,3005,060730133223005,Block 3005,G5040,U,78661.0,U,...,0,32.6288505,-116.9924214,4,22,"POLYGON ((-116.993114 32.628806, -116.993053 3...",15,24.0,0.003860,8.0
1957,6,73,13322,3000,060730133223000,Block 3000,G5040,U,78661.0,U,...,0,32.634625,-116.991596,162,575,"POLYGON ((-116.996325 32.633525, -116.996153 3...",15,5688.0,0.018311,238.0


### Drone Days / CFS in Block
- 792 days in analysis

In [38]:
from datetime import date
 
def numOfDays(date1, date2):
    if date2 > date1:   
        return (date2-date1).days
    else:
        return (date1-date2).days

start= date(2021, 7, 1)
end = date(2023, 9, 1)

days_in_analysis = numOfDays(start,end)

In [49]:
def calculate_summary(index):
    block = cv_blocks.iloc[index]
    drones_on_block = flight_details[flight_details['GEOID20']==block['GEOID20']] 
    calls_to_block = calls_for_service[calls_for_service['GEOID20']==block['GEOID20']] 

    block['num_flights'] = drones_on_block['id'].drop_duplicates().shape[0]
    block['flights_per_day'] = block['num_flights']/days_in_analysis
    block['num_calls'] = calls_to_block.shape[0]
    block['calls_per_day'] = block['num_calls']/days_in_analysis
    return block

In [55]:
from multiprocess.pool import Pool

with Pool(10) as pool:

    drones_cfs_pdpb = list(
        tqdm(pool.imap(calculate_summary, range(0, cv_blocks.shape[0])), total=cv_blocks.shape[0])
    )
    drones_cfs_pdpb = pd.DataFrame(drones_cfs_pdpb)

  0%|          | 0/1959 [00:00<?, ?it/s]

In [58]:
drones_cfs_pdpb.to_csv('../../data/outputs/output_cv-blocks-drones-calls.csv',index=False)

In [60]:
drones_cfs_pdpb.sort_values('num_flights').tail()

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry,len,num_flights,flights_per_day,num_calls,calls_per_day
1920,6,73,12700,3000,060730127003000,Block 3000,G5040,U,78661.0,U,...,32.6382813,-117.0855165,363,837,"POLYGON ((-117.088058 32.639014, -117.087241 3...",15,1784,2.252525,2384,3.010101
174,6,73,12302,1012,060730123021012,Block 1012,G5040,U,78661.0,U,...,32.6392976,-117.0811867,94,216,"POLYGON ((-117.083646 32.639302, -117.083247 3...",15,1937,2.445707,346,0.436869
387,6,73,13102,2001,060730131022001,Block 2001,G5040,U,78661.0,U,...,32.61484,-117.0718154,516,1466,"POLYGON ((-117.074527 32.615993, -117.074092 3...",15,2408,3.040404,422,0.532828
1223,6,73,13102,2000,060730131022000,Block 2000,G5040,U,78661.0,U,...,32.6182878,-117.0731641,326,862,"POLYGON ((-117.075881 32.619439, -117.075064 3...",15,2960,3.737374,417,0.526515
1787,6,73,12302,1013,060730123021013,Block 1013,G5040,U,78661.0,U,...,32.6399935,-117.0828161,130,307,"POLYGON ((-117.083982 32.640144, -117.082001 3...",15,3519,4.443182,65,0.082071


In [24]:
unique_days_in_block

,GEOID20,days
0,060730032041009,1
1,060730032041012,2
2,060730032041013,35
3,060730032041014,17
4,060730032041015,2
...,...,...
1610,060730219001089,4
1611,060730219001090,5
1612,060730219001091,8
1613,060730219001092,36
